In [ ]:
import csv
import numpy as np
import pandas as pd
import math
import seaborn as sb
from matplotlib import pyplot as plt
import trianglesolver as ts
from math import pi

In [ ]:
# d = previous state distance to target
# b = previous state bearing to target
# d_meas = new measurement of distance to target
# b_meas = new measurement of bearing to target

def kalman_update(d, b, d_meas, d_bear, K):
    d_prime = K * d + (1-K) * d_meas
    b_prime = K * b + (1-K) * d_bear
    return (d_prime, b_prime)

In [ ]:
def rerange(x):
    if (x >math.pi/2):
        x = x-math.pi
    return x

In [ ]:
# a = previous state distance to target
# B = previous state bearing to target
# c = amount moved forward
# Returns (b = new distance to target, A=new bearing to target)
# Note that if no motion, bearing and distance dont change

def kalman_predict(a, B, c):
    #print(f"KP {a} {B} {c}")
    if (c == 0):
        print("c == 0")
        return(a, B)
    elif (B == pi):
        print("B == pi")
        return(a, B)
    elif (B > pi):
        B_prime = (2*pi - B)
        print (f"B > pi: {B} {B_prime}")
        (tsa,tsb,tsc,tsA,tsB,tsC) = ts.solve(a=a, B=B_prime, c=c)
        return (tsb, 2*pi - tsA)
    else:
        (tsa,tsb,tsc,tsA,tsB,tsC) = ts.solve(a=a, B=B, c=c)
        return(tsb, tsA)

def another_kalman_predict(a, B, c):
    print(f"KP {a} {B} {c}")
    if (c == 0):
        return(a, B)
    else:
        b = law_of_cosines_b(a, c, B)
        A = law_of_sines_A(a,B,b)
    return (b, A)

def dumb_kalman_predict(d, b, m):
    d_prime = d
    b_prime = b
    return d_prime, b_prime

In [ ]:
# Edges are named after the opposite angle
def law_of_sines_A_actual(a, B, b):
    """ returns angle A given sides a,b and angle B"""
    temp = (a * math.sin(B)) / b
    if (temp < -1 or temp > 1.0):
        print("losA error", temp)
    return (math.asin((a * math.sin(B)) / b))

def law_of_sines_A(a, B, b):
    Btype = 'acute' if B < pi/2 else 'obtuse'
    a, b, c, A, B, C = ts.solve(a=a, B=B, b=b, ssa_flag=Btype)
    print (A)
    return A

In [ ]:
# Capital letter B is an angle. Small a, b and c are edges. 
# a is the edge across from A (I think that covers it.)
def law_of_cosines_B_actual(a, b, c):
    """ returns angle B given sides a,b and c"""
    temp = (b**2 - a**2 - c**2)/(-2.0 * a * c)
    if (temp < -1 or temp > 1.0):
        print("loc error")
    return math.acos((b**2 - a**2 - c**2)/(-2.0 * a * c))

def law_of_cosines_B(a, b, c):
    a, b, c, A, B, C = ts.solve(a=a, b=b, c=c)
    return B

In [ ]:
# Capital letter B is an angle. Small a, b and c are edges. 
# a is the edge across from A (I think that covers it.)
def law_of_cosines_b_actual(a, c, B):
    """ returns side b given sides a,c and angle B"""
    return(math.sqrt(a**2 + c**2 - 2*a*c*math.cos(B)))

def law_of_cosines_b(a, c, B):
    Btype = 'acute' if B < pi/2 else 'obtuse'
    if B==0 or a == 0 or c == 0:
        print(a, B, c)
    if B >= pi:
        print(f"B {B}")

    a, b, c, A, B, C = ts.solve(a=a, B=B, c=c, ssa_flag=Btype)
    return(b)

In [ ]:
# Capital letter A is an angle. Small a, b and c are edges. 
# a is the edge across from A (I think that covers it.)
def law_of_cosines_A_actual(a, b, c):
    """ returns angle A given sides a,b and angle B"""
    return math.acos((-(a*a) + b*b + c*c) / (2 * b * c))

In [ ]:
# elapsed,g_forward_cmd,g_turn_cmd,g_shortest_bearing,g_shortest
df=pd.read_csv('motion4.csv', sep=',')

In [ ]:
def test_harnass(K):
    state_dist = 0.8
    state_bear = math.radians(80)
    for index, row in df.iterrows():
        meas_dist = row['g_shortest']
        meas_bear = math.radians(row['g_shortest_bearing'])
        control_motion = row['g_forward_cmd'] * row['elapsed']
        temp_state_dist, temp_state_bear = kalman_predict(state_dist, state_bear, control_motion)
        pure_state_dist, pure_state_bear = kalman_predict(meas_dist, meas_bear, control_motion)
        state_dist, state_bear = kalman_update(temp_state_dist, temp_state_bear, meas_dist, meas_bear, K)
        df.loc[df.index[index], 'm'] = control_motion    
        df.loc[df.index[index], 'meas_dist'] = meas_dist
        df.loc[df.index[index], 'pure_state_dist'] = pure_state_dist    
        df.loc[df.index[index], 'temp_state_dist'] = temp_state_dist
        df.loc[df.index[index], 'state_dist'] = state_dist
        df.loc[df.index[index], 'meas_bear'] = math.degrees(meas_bear)
        df.loc[df.index[index], 'pure_state_bear'] = math.degrees(pure_state_bear)
        df.loc[df.index[index], 'state_bear'] = math.degrees(state_bear)
        df.loc[df.index[index], 'temp_state_bear'] = math.degrees(temp_state_bear)
        df.loc[df.index[index], 'K'] = K
    return df

In [ ]:
df = test_harnass(0.6)
df.plot(y=['state_dist', 'meas_dist'], figsize=[14,4])
df.plot(y=['state_bear', 'meas_bear'], figsize=[14,4])

In [ ]:
# Test various trig functions
from math import pi
import trianglesolver as ts
ts.solve(a=1, b=1, c=1)

df1 = pd.DataFrame(columns = ["b_index", "B", "l", "d", "d_prime", "b_prime"])
start = 1
end = 360
for b_index in range (start, end):
    B = math.radians(b_index)
    l = 1
    d = 1
    d_prime, b_prime = kalman_predict(d, B, l)
    df1 = df1.append([{"B": math.degrees(B), 
                     "l":l,
                     "d": d, 
                     "d_prime": d_prime, 
                     "b_prime": math.degrees(b_prime), 
                     "b_index": b_index}])
df1 = df1.set_index("b_index")    

In [ ]:
df1.plot()